In [116]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
#from sklearn.preprocessing import PolynomialFeatures
from multiprocessing import Pool
import multiprocessing
import ZVnbrosse
from sklearn.preprocessing import PolynomialFeatures
from potentials import GaussPotential,GaussMixture,GausMixtureIdent,GausMixtureSame,BananaShape
from samplers import MCMC_sampler,Generate_train,ULA_light
from baselines import set_function,construct_ESVM_kernel,GenerateSigma
from martingale import approx_q
from optimize import Run_eval_test,optimize_parallel_new 
from utils import * 
import copy

In [117]:
def H(k, x):
    if k==0:
        return 1.0
    if k ==1:
        return x
    if k==2:
        return (x**2 - 1)/np.sqrt(2)
    c = np.zeros(k+1,dtype = float)
    c[k] = 1.0
    h = P.hermite_e.hermeval(x,c) / np.sqrt(sp.special.factorial(k)) 
    return h

In [118]:
def test_traj(coefs_poly_regr,gamma,r_seed,lag,K_max,S_max,N_test,f_type,x0):
    """
    function to perform 1-dimensional martingale decomposition
    """
    X_test,Noise = generate_traj(x0,N_test,gamma,r_seed)
    test_stat_vanilla = np.zeros(N_test,dtype = float)
    test_stat_vr = np.zeros_like(test_stat_vanilla)
    #compute number of basis polynomials
    num_basis_funcs = K_max+1
    #compute polynomials of noise variables Z_l
    poly_vals = np.zeros((num_basis_funcs,N_test), dtype = float)
    for k in range(len(poly_vals)):
        poly_vals[k,:] = H(k,Noise)
    #initialize function
    f_vals_vanilla = X_test
    #array to store control variates values
    cvfs = np.zeros_like(f_vals_vanilla)
    #compute coeffitients bar_a
    bar_a_1 = np.zeros((lag,N_test),dtype=float)
    bar_a_2 = np.zeros_like(bar_a_1)
    alpha = 1./4*gamma*(sig**2)*(1./(1-gamma*a) + 1.)
    beta = gamma*(a*c-1./2*sig**2)
    for i in range(lag):
        #second-order coefficients
        #bar_a_2[i,1:] = coefs_poly_regr[i,2]*np.sqrt(2)*gamma*(sigma(X_test[:-1]))**2
        #bar_a_2[i,0] = coefs_poly_regr[i,2]*np.sqrt(2)*gamma*(sigma(x0))**2
        bar_a_2[i,1:] = coefs_poly_regr[i,1]*np.sqrt(2)*alpha +\
                    coefs_poly_regr[i,2]*np.sqrt(2)*(7*alpha**2 + gamma*(sigma(X_test[:-1]))**2)
        bar_a_2[i,0] = coefs_poly_regr[i,1]*np.sqrt(2)*alpha +\
                    coefs_poly_regr[i,2]*np.sqrt(2)*(7*alpha**2 + gamma*(sigma(x0))**2)
        #first-order coefficients
        #bar_a_1[i,1:] = coefs_poly_regr[i,1]*np.sqrt(gamma)*sigma(X_test[:-1]) +\
                #coefs_poly_regr[i,2]*2*np.sqrt(gamma)*sigma(X_test[:-1])*(X_test[:-1]+gamma*b(X_test[:-1]))
        bar_a_1[i,1:] = coefs_poly_regr[i,1]*np.sqrt(gamma)*sigma(X_test[:-1]) +\
                coefs_poly_regr[i,2]*(2*((1-gamma*a)*X_test[:-1] + beta)*(np.sqrt(gamma)*sigma(X_test[:-1]) + alpha) +\
                                    6*alpha*sigma(X_test[:-1])*np.sqrt(gamma))
        bar_a_1[i,0] = coefs_poly_regr[i,1]*np.sqrt(gamma)*sigma(x0) +\
                coefs_poly_regr[i,2]*(2*((1-gamma*a)*x0 + beta)*(np.sqrt(gamma)*sigma(x0) + alpha) +\
                                    6*alpha*sigma(x0)*np.sqrt(gamma))        
        #bar_a_1[i,0] = coefs_poly_regr[i,1]*np.sqrt(gamma)*sigma(x0) +\
                        #coefs_poly_regr[i,2]*2*np.sqrt(gamma)*sigma(x0)*(x0+gamma*b(x0))
    bar_a_1 = bar_a_1*poly_vals[1,:]
    bar_a_2 = bar_a_2*poly_vals[2,:]
    #compute martingale sums
    M_n_1 = 0.0
    M_n_2 = 0.0
    for l in range(N_test):
        for r in range(min(N_test-l,lag)):
            M_n_1 += bar_a_1[r,l]
            M_n_2 += bar_a_2[r,l]
    print("M_n_2: ",M_n_2)
    print("M_n_1: ",M_n_1)
    return np.mean(f_vals_vanilla),np.mean(f_vals_vanilla)-M_n_1/N_test,np.mean(f_vals_vanilla)-M_n_1/N_test-M_n_2/N_test

In [119]:
def approx_q(X_train,Y_train,N_traj_train,lag,max_deg):
    """
    Function to regress q functions on a polynomial basis;
    Args:
        X_train - train tralectory;
        Y_train - function values;
        N_traj_train - number of training trajectories;
        lag - truncation point for coefficients, those for |p-l| > lag are set to 0;
        max_deg - maximum degree of polynomial in regression
    """
    dim = X_train[0,:].shape[0]
    print("dimension = ",dim)
    coefs_poly = np.array([])
    for i in range(lag):
        x_all = np.array([])
        y_all = np.array([])
        for j in range(N_traj_train):
            y = Y_train[j,i:,0]
            if i == 0:
                x = X_train[j,:]
            else:
                x = X_train[j,:-i]
            #concatenate results
            if x_all.size == 0:
                x_all = x
            else:
                x_all = np.concatenate((x_all,x),axis = 0)
            y_all = np.concatenate([y_all,y])
        #should use polyfeatures here
        print("variance: ",np.var(y_all))
        print(y_all[:50])
        poly = PolynomialFeatures(max_deg)
        X_features = poly.fit_transform(x_all)
        print(X_features.shape)
        lstsq_results = np.linalg.lstsq(X_features,y_all,rcond = None)
        coefs = copy.deepcopy(lstsq_results[0])
        coefs.resize((1,X_features.shape[1]))           
        if coefs_poly.size == 0:
            coefs_poly = copy.deepcopy(coefs)
        else:
            coefs_poly = np.concatenate((coefs_poly,coefs),axis=0)
    return coefs_poly

In [120]:
a = 5.0
c = 5.0
sig = 2.0

def b(X_t):
    """
    b function in the diffusion
    """
    return a*(c-X_t)

def sigma(X_t):
    """
    b function in the diffusion
    """
    return sig*np.sqrt(X_t)

def sample_discretized_diffusion(X_t,gamma_t):
    """
    args:   X_t - current value, 
            gamma_t - step size;
    returns:  (X_{t+1},xi_{t+1}) - value at the next time moment and the corresponding noise variable
    """
    xi = np.random.randn()
    return (1-gamma_t*a)*X_t + gamma_t*(a*c-1./4*(sig**2)) + np.sqrt(gamma_t*X_t)*sig*xi + 0.25*gamma_t*(sig**2)*(xi**2 + (xi**2)/(1-gamma_t*a) - 1),xi

#currently we use this function without the burn-in
def generate_traj(x0,n,gamma,r_seed):
    """
    args:
        x0 - starting point;
        n - number of steps;
        gamma - step size (assumed to be fixed for now);
    returns:
        x_all,noise_all - np.arrays of shape (n,)  
    """
    x_all = np.zeros(n,dtype = float)
    noise_all = np.zeros(n,dtype = float)
    np.random.seed(r_seed)
    x_all[0],noise_all[0] = sample_discretized_diffusion(x0,gamma)
    for i in range(1,n):
        x_all[i],noise_all[i] = sample_discretized_diffusion(x_all[i-1],gamma)
    return x_all,noise_all

def run_monte_carlo(x,f_type):
    if f_type == "quadratic":
        f_vals = x**2
    else:
        raise "not implemented error"
    return np.mean(f_vals,axis=1)

In [121]:
n = 2*10**4 #sample size
gamma = 1e-2 # Step size
n_traj = 100
n_traj_test = 100 # Number of independent MCMC trajectories for test
f_type = "linear"
K_max = 2 #max degree of Hermite polynomial
S_max = 2 #max degree of polynomial during regression stage
lag = 100 #maximal lag order
N_test = 2*10**3

Sample discretized diffusion

In [122]:
def generate_observations(x0,n,gamma,r_seed,n_traj):
    X_train_all = np.zeros((n_traj,n,1),dtype=float)
    Noise_train_all = np.zeros_like(X_train_all)
    Y_train_all = np.zeros((n_traj,n,1),dtype=float)
    for i in range(n_traj):
        X_train, noise_train = generate_traj(x0,n,gamma,r_seed+i)
        #set target function
        Y_train = X_train
        X_train_all[i,:,0] = X_train
        Y_train_all[i,:,0] = Y_train
        Noise_train_all[i,:,0] = noise_train
        #X_train = X_train.reshape((1,-1,d))
        #Y_train = Y_train.reshape((1,-1,1))
    return X_train_all, Noise_train_all, Y_train_all

In [123]:
x0 = 1
r_seed = 1812
X_train, noise_train, Y_train = generate_observations(x0,n,gamma,r_seed,n_traj)
X_train_1, noise_train_1, Y_train_1 = generate_observations(x0,n,2*gamma,r_seed,n_traj)
X_train_2, noise_train_2, Y_train_2 = generate_observations(x0,n,3*gamma,r_seed,n_traj)
X_train_3, noise_train_3, Y_train_3 = generate_observations(x0,n,4*gamma,r_seed,n_traj)
X_train_4, noise_train_4, Y_train_4 = generate_observations(x0,n,5*gamma,r_seed,n_traj)
#set target function
#X_train = X_train.reshape((1,-1,1))
#Y_train = Y_train.reshape((1,-1,1))

### Bernoulli:: Optimize coefficients by solving regression with polynomial features

In [ ]:
#polynomial coefficients
coefs_poly = approx_q(X_train,Y_train,n_traj,lag,S_max)
coefs_poly_1 = approx_q(X_train_1,Y_train_1,n_traj,lag,S_max)
coefs_poly_2 = approx_q(X_train_2,Y_train_2,n_traj,lag,S_max)
coefs_poly_3 = approx_q(X_train_3,Y_train_3,n_traj,lag,S_max)
coefs_poly_4 = approx_q(X_train_4,Y_train_4,n_traj,lag,S_max)
print(coefs_poly.shape)
print(coefs_poly)
regr_vals = np.zeros((lag,X_train.shape[1]),dtype=float)
for i in range(len(regr_vals)):
    for j in range(S_max+1):
        regr_vals[i,:] += coefs_poly[i,j]*X_train[0,:,0]**j

dimension =  20000
variance:  2.0899943697900487
[1.11285866 1.2255869  1.55271243 1.81105758 1.75029627 1.65280518
 2.21387003 2.74105429 2.58837509 3.0605777  3.66908742 3.43003029
 3.26323434 3.48087211 3.68928895 3.07141543 2.75347673 2.78328609
 2.99254283 3.23248039 3.48558467 3.48161464 3.52823879 3.74045447
 3.79148263 4.42797009 4.4768625  4.86199417 4.83148923 5.14511952
 4.80503389 4.43393323 4.54337947 4.64258046 4.56072147 5.34903561
 5.94173732 5.60456608 4.95761333 4.05923911 3.53537388 4.22319737
 4.40223358 4.57778945 4.54046422 4.05443041 3.76322632 3.58416409
 3.09853922 3.60274567]
(2000000, 3)
variance:  2.0893644516329144
[1.2255869  1.55271243 1.81105758 1.75029627 1.65280518 2.21387003
 2.74105429 2.58837509 3.0605777  3.66908742 3.43003029 3.26323434
 3.48087211 3.68928895 3.07141543 2.75347673 2.78328609 2.99254283
 3.23248039 3.48558467 3.48161464 3.52823879 3.74045447 3.79148263
 4.42797009 4.4768625  4.86199417 4.83148923 5.14511952 4.80503389
 4.43393323 4

variance:  2.085588841373846
[3.68928895 3.07141543 2.75347673 2.78328609 2.99254283 3.23248039
 3.48558467 3.48161464 3.52823879 3.74045447 3.79148263 4.42797009
 4.4768625  4.86199417 4.83148923 5.14511952 4.80503389 4.43393323
 4.54337947 4.64258046 4.56072147 5.34903561 5.94173732 5.60456608
 4.95761333 4.05923911 3.53537388 4.22319737 4.40223358 4.57778945
 4.54046422 4.05443041 3.76322632 3.58416409 3.09853922 3.60274567
 3.16179801 3.24660245 3.16090388 3.60372309 3.79587744 4.13814625
 4.40957779 4.53668206 4.14375436 3.89963825 4.22899249 4.6076403
 4.22215846 5.2695754 ]
(1998600, 3)
variance:  2.0854790033487
[3.07141543 2.75347673 2.78328609 2.99254283 3.23248039 3.48558467
 3.48161464 3.52823879 3.74045447 3.79148263 4.42797009 4.4768625
 4.86199417 4.83148923 5.14511952 4.80503389 4.43393323 4.54337947
 4.64258046 4.56072147 5.34903561 5.94173732 5.60456608 4.95761333
 4.05923911 3.53537388 4.22319737 4.40223358 4.57778945 4.54046422
 4.05443041 3.76322632 3.58416409 3.09

variance:  2.0848522497758704
[4.83148923 5.14511952 4.80503389 4.43393323 4.54337947 4.64258046
 4.56072147 5.34903561 5.94173732 5.60456608 4.95761333 4.05923911
 3.53537388 4.22319737 4.40223358 4.57778945 4.54046422 4.05443041
 3.76322632 3.58416409 3.09853922 3.60274567 3.16179801 3.24660245
 3.16090388 3.60372309 3.79587744 4.13814625 4.40957779 4.53668206
 4.14375436 3.89963825 4.22899249 4.6076403  4.22215846 5.2695754
 5.68227652 5.85166262 6.32580024 5.68009219 4.9916804  4.32062711
 3.61349588 3.48803901 3.10310745 3.47573396 2.96675966 3.28873412
 3.69438994 3.65570201]
(1997200, 3)
variance:  2.0848475614939326
[5.14511952 4.80503389 4.43393323 4.54337947 4.64258046 4.56072147
 5.34903561 5.94173732 5.60456608 4.95761333 4.05923911 3.53537388
 4.22319737 4.40223358 4.57778945 4.54046422 4.05443041 3.76322632
 3.58416409 3.09853922 3.60274567 3.16179801 3.24660245 3.16090388
 3.60372309 3.79587744 4.13814625 4.40957779 4.53668206 4.14375436
 3.89963825 4.22899249 4.6076403 

variance:  2.0849384543958123
[4.40223358 4.57778945 4.54046422 4.05443041 3.76322632 3.58416409
 3.09853922 3.60274567 3.16179801 3.24660245 3.16090388 3.60372309
 3.79587744 4.13814625 4.40957779 4.53668206 4.14375436 3.89963825
 4.22899249 4.6076403  4.22215846 5.2695754  5.68227652 5.85166262
 6.32580024 5.68009219 4.9916804  4.32062711 3.61349588 3.48803901
 3.10310745 3.47573396 2.96675966 3.28873412 3.69438994 3.65570201
 3.65446468 4.00575817 4.51569429 5.10326791 5.5888181  5.52673195
 4.847571   5.35118168 5.26188606 4.7813472  5.13062099 5.34410156
 4.73453183 4.80036935]
(1995800, 3)
variance:  2.084952125005967
[4.57778945 4.54046422 4.05443041 3.76322632 3.58416409 3.09853922
 3.60274567 3.16179801 3.24660245 3.16090388 3.60372309 3.79587744
 4.13814625 4.40957779 4.53668206 4.14375436 3.89963825 4.22899249
 4.6076403  4.22215846 5.2695754  5.68227652 5.85166262 6.32580024
 5.68009219 4.9916804  4.32062711 3.61349588 3.48803901 3.10310745
 3.47573396 2.96675966 3.28873412

variance:  2.085182629350034
[4.40957779 4.53668206 4.14375436 3.89963825 4.22899249 4.6076403
 4.22215846 5.2695754  5.68227652 5.85166262 6.32580024 5.68009219
 4.9916804  4.32062711 3.61349588 3.48803901 3.10310745 3.47573396
 2.96675966 3.28873412 3.69438994 3.65570201 3.65446468 4.00575817
 4.51569429 5.10326791 5.5888181  5.52673195 4.847571   5.35118168
 5.26188606 4.7813472  5.13062099 5.34410156 4.73453183 4.80036935
 4.56659413 4.36950942 5.27714351 5.27627046 6.88350413 7.56710948
 8.76101257 9.11841857 8.18042846 8.21735549 8.22408228 8.10194333
 7.71326233 6.83017653]
(1994400, 3)
variance:  2.0851836938693173
[4.53668206 4.14375436 3.89963825 4.22899249 4.6076403  4.22215846
 5.2695754  5.68227652 5.85166262 6.32580024 5.68009219 4.9916804
 4.32062711 3.61349588 3.48803901 3.10310745 3.47573396 2.96675966
 3.28873412 3.69438994 3.65570201 3.65446468 4.00575817 4.51569429
 5.10326791 5.5888181  5.52673195 4.847571   5.35118168 5.26188606
 4.7813472  5.13062099 5.34410156 4

variance:  2.085017171099553
[3.61349588 3.48803901 3.10310745 3.47573396 2.96675966 3.28873412
 3.69438994 3.65570201 3.65446468 4.00575817 4.51569429 5.10326791
 5.5888181  5.52673195 4.847571   5.35118168 5.26188606 4.7813472
 5.13062099 5.34410156 4.73453183 4.80036935 4.56659413 4.36950942
 5.27714351 5.27627046 6.88350413 7.56710948 8.76101257 9.11841857
 8.18042846 8.21735549 8.22408228 8.10194333 7.71326233 6.83017653
 7.2624782  7.62852472 6.41637274 6.10107382 5.69282478 5.99242579
 5.83849256 5.9422928  6.06369453 6.55412972 6.51933563 6.71758233
 6.53654882 6.36334616]
(1993000, 3)
variance:  2.084990321311512
[3.48803901 3.10310745 3.47573396 2.96675966 3.28873412 3.69438994
 3.65570201 3.65446468 4.00575817 4.51569429 5.10326791 5.5888181
 5.52673195 4.847571   5.35118168 5.26188606 4.7813472  5.13062099
 5.34410156 4.73453183 4.80036935 4.56659413 4.36950942 5.27714351
 5.27627046 6.88350413 7.56710948 8.76101257 9.11841857 8.18042846
 8.21735549 8.22408228 8.10194333 7.

variance:  2.0847461447103663
[4.847571   5.35118168 5.26188606 4.7813472  5.13062099 5.34410156
 4.73453183 4.80036935 4.56659413 4.36950942 5.27714351 5.27627046
 6.88350413 7.56710948 8.76101257 9.11841857 8.18042846 8.21735549
 8.22408228 8.10194333 7.71326233 6.83017653 7.2624782  7.62852472
 6.41637274 6.10107382 5.69282478 5.99242579 5.83849256 5.9422928
 6.06369453 6.55412972 6.51933563 6.71758233 6.53654882 6.36334616
 6.15980173 6.18508149 5.82775675 6.16278455 6.78916086 6.70931175
 7.30159501 6.16342321 5.97116848 5.55887301 5.62477325 6.45366712
 6.16636023 6.15684086]
(1991600, 3)
variance:  2.0847420867069553
[5.35118168 5.26188606 4.7813472  5.13062099 5.34410156 4.73453183
 4.80036935 4.56659413 4.36950942 5.27714351 5.27627046 6.88350413
 7.56710948 8.76101257 9.11841857 8.18042846 8.21735549 8.22408228
 8.10194333 7.71326233 6.83017653 7.2624782  7.62852472 6.41637274
 6.10107382 5.69282478 5.99242579 5.83849256 5.9422928  6.06369453
 6.55412972 6.51933563 6.71758233

variance:  2.0847831764602303
[8.76101257 9.11841857 8.18042846 8.21735549 8.22408228 8.10194333
 7.71326233 6.83017653 7.2624782  7.62852472 6.41637274 6.10107382
 5.69282478 5.99242579 5.83849256 5.9422928  6.06369453 6.55412972
 6.51933563 6.71758233 6.53654882 6.36334616 6.15980173 6.18508149
 5.82775675 6.16278455 6.78916086 6.70931175 7.30159501 6.16342321
 5.97116848 5.55887301 5.62477325 6.45366712 6.16636023 6.15684086
 5.06144319 4.74517973 4.78706468 4.7116949  4.92264644 4.21730541
 4.2744425  4.43642412 4.53026598 4.94886749 5.44168319 5.60280267
 5.1569707  4.64337089]
(1990200, 3)
variance:  2.084780461911094
[9.11841857 8.18042846 8.21735549 8.22408228 8.10194333 7.71326233
 6.83017653 7.2624782  7.62852472 6.41637274 6.10107382 5.69282478
 5.99242579 5.83849256 5.9422928  6.06369453 6.55412972 6.51933563
 6.71758233 6.53654882 6.36334616 6.15980173 6.18508149 5.82775675
 6.16278455 6.78916086 6.70931175 7.30159501 6.16342321 5.97116848
 5.55887301 5.62477325 6.45366712

variance:  2.1477363047670277
[4.36518304 4.63676007 4.88332638 3.80829197 3.2809496  3.31672777
 3.62914528 3.97680443 4.33139111 4.2644067  4.28342413 4.56400566
 4.58018626 5.5295756  5.5003814  6.0118344  5.84329548 6.23002135
 5.58541657 4.94470921 5.0619422  5.16018258 4.9856945  6.14423383
 6.96337937 6.31702685 5.26980621 3.9785352  3.29884637 4.33018098
 4.58844239 4.8306806  4.75187587 4.04781937 3.66201119 3.45233037
 2.85540811 3.63120566 3.05682535 3.22909081 3.15239979 3.83382037
 4.11972483 4.61944859 4.99262806 5.13117023 4.493125   4.11625509
 4.59868771 5.1383054 ]
(1998800, 3)
variance:  2.1477329225906083
[4.63676007 4.88332638 3.80829197 3.2809496  3.31672777 3.62914528
 3.97680443 4.33139111 4.2644067  4.28342413 4.56400566 4.58018626
 5.5295756  5.5003814  6.0118344  5.84329548 6.23002135 5.58541657
 4.94470921 5.0619422  5.16018258 4.9856945  6.14423383 6.96337937
 6.31702685 5.26980621 3.9785352  3.29884637 4.33018098 4.58844239
 4.8306806  4.75187587 4.0478193

Test our regressors

In [ ]:
cur_lag = 0
N_pts = 500
plt.figure(figsize=(10, 10))
plt.title("Testing regression model",fontsize=20)
plt.plot(Y_train[0,cur_lag:N_pts+cur_lag,0],color='r',label='true function')
plt.plot(regr_vals[cur_lag,:N_pts],color='g',label = 'practical approximation')
plt.legend(loc = 'upper left',fontsize = 16)
plt.show()

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(test_traj, [(coefs_poly,gamma,test_seed+i,lag,K_max,S_max,N_test,f_type,x0) for i in range (n_traj_test)])
#res = trav.starmap(test_traj, [(Cur_pot,coefs_poly,step,test_seed+i,lag,K_max,S_max,N_burn,N_test,d,f_type,inds_arr,params,x0,fixed_start) for i in range (n_traj_test)])
trav.close()
res = np.asarray(res)

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res_1 = trav.starmap(test_traj, [(coefs_poly_1,2*gamma,test_seed+i,lag,K_max,S_max,N_test,f_type,x0) for i in range (n_traj_test)])
#res = trav.starmap(test_traj, [(Cur_pot,coefs_poly,step,test_seed+i,lag,K_max,S_max,N_burn,N_test,d,f_type,inds_arr,params,x0,fixed_start) for i in range (n_traj_test)])
trav.close()
res_1 = np.asarray(res_1)

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res_2 = trav.starmap(test_traj, [(coefs_poly_2,3*gamma,test_seed+i,lag,K_max,S_max,N_test,f_type,x0) for i in range (n_traj_test)])
#res = trav.starmap(test_traj, [(Cur_pot,coefs_poly,step,test_seed+i,lag,K_max,S_max,N_burn,N_test,d,f_type,inds_arr,params,x0,fixed_start) for i in range (n_traj_test)])
trav.close()
res_2 = np.asarray(res_2)

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res_3 = trav.starmap(test_traj, [(coefs_poly_3,4*gamma,test_seed+i,lag,K_max,S_max,N_test,f_type,x0) for i in range (n_traj_test)])
#res = trav.starmap(test_traj, [(Cur_pot,coefs_poly,step,test_seed+i,lag,K_max,S_max,N_burn,N_test,d,f_type,inds_arr,params,x0,fixed_start) for i in range (n_traj_test)])
trav.close()
res_3 = np.asarray(res_3)

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res_4 = trav.starmap(test_traj, [(coefs_poly_4,5*gamma,test_seed+i,lag,K_max,S_max,N_test,f_type,x0) for i in range (n_traj_test)])
#res = trav.starmap(test_traj, [(Cur_pot,coefs_poly,step,test_seed+i,lag,K_max,S_max,N_burn,N_test,d,f_type,inds_arr,params,x0,fixed_start) for i in range (n_traj_test)])
trav.close()
res_4 = np.asarray(res_4)

### Comparison plots

In [ ]:
def set_axis_style_boxplot(ax, labels, parts):
    colors = (sns.color_palette("muted")[0:7])
    ax.grid(color='black', linestyle='-', linewidth=0.15, alpha=0.6)    
    ax.set_xticks(np.arange(1, len(labels)+1))
    ax.set_xticklabels(labels, fontsize=12)
    ax.set_xlim(0.5, len(labels) + 0.5)
    #ax.set_ylim(-0.12, 0.12)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    
    for pc,i in zip(parts['boxes'],range(len(labels))):
        pc.set(facecolor=colors[i],alpha=0.65)
        pc.set_edgecolor('black')
        pc.set_linewidth(0.65)
    

def boxplot_ind(data, title, labels, path):
    meanprops = dict(linestyle='-', linewidth=1, color='black')
    medianprops = dict(linestyle='', linewidth=0)

    fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8, 4), sharey=True, frameon=False,dpi=100)      
    fig.suptitle(title, fontsize=20)
    parts = ax1.boxplot(data,  widths=0.6, patch_artist=True, meanline=True, showmeans=True, medianprops=medianprops,meanprops = meanprops, showfliers=False)
    set_axis_style_boxplot(ax1, labels, parts)
    fig.tight_layout()
    fig.subplots_adjust(top=0.85)
    plt.savefig(path)
    plt.show() 

In [ ]:
#data = [res_new[:,0],res_new[:,1],res_new[:,2]] 
#boxplot_ind(data, title, labels,path="./diffusion_quadratic_09_11.pdf")
title = ""
#labels = ['Vanilla\n Euler scheme', 'Euler scheme \nwith MDCV-1']
labels = ['MDCV-2, \n $\\gamma = 0.01$', 'MDCV-2, \n $\\gamma = 0.02$',\
          'MDCV-2, \n $\\gamma = 0.03$', 'MDCV-2, \n $\\gamma = 0.04$', 'MDCV-2, \n $\\gamma = 0.05$']#,\

In [ ]:
data = [res[:,2],res_1[:,2],res_2[:,2], res_3[:,2],res_4[:,2]]#,res_new_4[:,2]] 
#data = [res_new_1[:,2], res_new_2[:,2],res_new_3[:,2],res_new_4[:,2]] 
boxplot_ind(data, title, labels,path="./CIR_gamma_influence.pdf")